
<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning">
</div>


# Populating the Metastore
In this demo, we will populate the metastore, focusing on the three-level namespace concept to create data containers and objects. We will cover the creation and management of catalogs, schemas, tables, views, and user-defined functions, demonstrating the execution of SQL commands to achieve these tasks. Additionally, we will verify the settings and inspect the data objects to ensure they are correctly stored and accessible for further analysis. This process includes creating and populating a managed table and view, as well as defining and executing a user-defined function to enhance SQL capabilities within the Unity Catalog environment.

### Learning Objectives
By the end of this demo, you will be able to:
1. Identify the prerequisites and understand the three-level namespace concept introduced by Unity Catalog.
2. Describe the process of creating and managing catalogs, schemas, tables, views, and user-defined functions within Unity Catalog.
3. Execute SQL commands to create and manage catalogs, schemas, tables, and views, and implement user-defined functions in the Unity Catalog environment.
4. Inspect and verify the current catalog and schema settings, ensuring that the objects created are correctly stored and accessible for further analysis.
5. Develop and populate a managed table and a view, as well as define and execute a user-defined function to extend SQL capabilities within the Unity Catalog.

## REQUIRED - SELECT CLASSIC COMPUTE

Before executing cells in this notebook, please select your classic compute cluster in the lab. Be aware that **Serverless** is enabled by default.

Follow these steps to select the classic compute cluster:


1. Navigate to the top-right of this notebook and click the drop-down menu to select your cluster. By default, the notebook will use **Serverless**.

2. If your cluster is available, select it and continue to the next cell. If the cluster is not shown:

   - Click **More** in the drop-down.

   - In the **Attach to an existing compute resource** window, use the first drop-down to select your unique cluster.

**NOTE:** If your cluster has terminated, you might need to restart it in order to select it. To do this:

1. Right-click on **Compute** in the left navigation pane and select *Open in new tab*.

2. Find the triangle icon to the right of your compute cluster name and click it.

3. Wait a few minutes for the cluster to start.

4. Once the cluster is running, complete the steps above to select your cluster.

## A. Classroom Setup

Run the following cell to configure your working environment for this course. 

**NOTE:** The `DA` object is only used in Databricks Academy courses and is not available outside of these courses. It will dynamically reference the information needed to run the course.

In [0]:
%run ./Includes/Classroom-Setup-1

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


Your Unity Catalog name:,


In this course, we may use dynamically generated variables created by the classroom setup scripts to reference your catalog and/or schema within the `DA` object. For example, the `DA.catalog_name` variable will dynamically reference your specific catalog when executing SQL code.


Run the cell below to view the value of the variable and confirm it matches your catalog name in the cell above.

In [0]:
%python
print(DA.catalog_name)

labuser11086062_1754032747


## B. Three-Level Namespace Recap



#### B1. Traditional Two-Level Namespace

Anyone with SQL experience will likely be familiar with the traditional two-level namespace to address tables or views within a schema (often referred to as a database) as shown in the following example:
<br></br>

```sql
SELECT * 
FROM myschema.mytable;
```
<br>

#### B2. Unity Catalog Three-Level Namespace
Unity Catalog introduces the concept of a **catalog** into the hierarchy, which provides another containment layer above the schema layer. This provides a new way for organizations to segregate their data and can be handy in many use cases. For example:

* Separating data relating to business units within your organization (sales, marketing, human resources, etc)
* Satisfying SDLC requirements (dev, staging, prod, etc)
* Establishing sandboxes containing temporary datasets for internal use

You can have as many catalogs as you want in the metastore, and each can contain as many schemas as you want. To deal with this additional level, complete table/view references in Unity Catalog use a three-level namespace, like this:
<br></br>
```sql
SELECT * 
FROM mycatalog.myschema.mytable;
```
We can take advantage of the **`USE`** statements to select a default catalog or schema to make our queries easier to write and read:
<br></br>
```sql
USE CATALOG mycatalog;
USE SCHEMA myschema;
```

## C. Create Data Containers and Objects to Populate the Metastore

In this section, let's explore how to create data containers and objects in the metastore. This can be done using SQL and Python. We will focus on SQL in this course. 

  Note that the SQL statements used throughout this lab could also be applied to a DBSQL warehouse as well.

### C1. Catalogs

#### C1.1 Create a Catalog 

1. To create a catalog, use the `CREATE CATALOG <catalog_name>` statement. The `IF NOT EXISTS` keywords will prevent any errors if the catalog name already exists.

##### NOTE - PLEASE READ, CELL BELOW WILL RETURN AN ERROR (permission denied)
The code is for example purposes only. Your lab environment does not grant you permission to create your own catalog in this shared training workspace. Instead, the training environment has automatically created your course catalog for you.

Be aware, depending on your organization's setup, there may be occasions when you do not have permission to create catalogs in your environment.

In [0]:
-------------------------------------
-- Note below will fail on purpose --
-------------------------------------
CREATE CATALOG IF NOT EXISTS the_catalog_name;

com.databricks.sql.managedcatalog.acl.UnauthorizedAccessException: PERMISSION_DENIED: User does not have CREATE CATALOG on Metastore '3686620-us-west-2'.
	at com.databricks.managedcatalog.ErrorDetailsHandler.wrapServiceException(ErrorDetailsHandler.scala:87)
	at com.databricks.managedcatalog.ErrorDetailsHandler.wrapServiceException$(ErrorDetailsHandler.scala:63)
	at com.databricks.managedcatalog.ManagedCatalogClientImpl.wrapServiceException(ManagedCatalogClientImpl.scala:251)
	at com.databricks.managedcatalog.ManagedCatalogClientImpl.recordAndWrapExceptionBase(ManagedCatalogClientImpl.scala:7377)
	at com.databricks.managedcatalog.ManagedCatalogClientImpl.recordAndWrapException(ManagedCatalogClientImpl.scala:7363)
	at com.databricks.managedcatalog.ManagedCatalogClientImpl.createCatalogProto(ManagedCatalogClientImpl.scala:505)
	at com.databricks.managedcatalog.ManagedCatalogClientImpl.$anonfun$createCatalog$9(ManagedCatalogClientImpl.scala:464)
	at com.databricks.managedcatalog.ManagedCa

#### C1.2 Selecting a Default Catalog

1. Let's set **your** specific course catalog as the default catalog using the `USE CATALOG <catalog_name>` statement.

   After this, any schema references you make will be assumed to be in this catalog unless you explicitly select a different catalog using a three-level specification.

   **NOTE:** The cell below uses `spark.sql()` to dynamically specify the catalog name using the `DA.catalog_name` variable.

In [0]:
%python
spark.sql(f'USE CATALOG {DA.catalog_name}')

DataFrame[]

#### C1.3 Verify the Current Catalog
1. Inspect the current catalog setting again using the `current_catalog()` function in SQL. Notice that your default catalog has been set to your specific catalog name.

In [0]:
SELECT current_catalog()

current_catalog()
labuser11086062_1754032747


### C2. Schemas

#### C2.1 Creating and Using a Schema
1. In the following code cell, we will:
    * Create a schema named **example** within our default catalog set in the previous cell
    * Select the **example** schema as the default schema
    * Verify that our default schema is, indeed, the **example**

**NOTE:** The concept of a schema in Unity Catalog is similar to the schemas or databases with which you're already likely familiar. Schemas contain data objects like tables and views but can also contain functions and ML/AI models. Let's create a schema in the catalog we just created. We don't have to worry about name uniqueness since our new catalog represents a clean namespace for the schemas it can hold.


In [0]:
-- Create a new schema
CREATE SCHEMA IF NOT EXISTS example;

-- Make our newly created schema the default
USE SCHEMA example;

-- Verify the current default schema
SELECT current_catalog(), current_schema();

current_catalog(),current_schema()
labuser11086062_1754032747,example


2. Manually view the catalog and schema using the navigation bar on the left by completing the following steps:
   - Select the catalog icon on the left.

   - Expand your unique catalog name.

   - Expand the **example** schema we previously created.

   - Leave your catalog explorer open on the left.

### C3. Managed Tables
With all the necessary containers in place, let's turn our attention to creating some data objects.

#### C3.1 Create a Managed Table

1. First let's create a table named **silver**. For this example, we'll pre-populate the table with a simple mock dataset containing synthetic patient heart rate data.

    Note the following:
    * We only need to specify the table name when creating the table. We don't need to specify the containing catalog or schema because we already selected defaults earlier with the `USE` statements.
    * This will be a Managed table since we aren't specifying a `LOCATION` or `PATH` keyword in `CREATE TABLE` statement.
    * Because it's a Managed table, it must be a Delta Lake table. (as opposed to Parquet, AVRO, ORC, etc.)

In [0]:
-- Create an empty silver table
CREATE OR REPLACE TABLE silver (
  device_id  INT,
  mrn        STRING,
  name       STRING,
  time       TIMESTAMP,
  heartrate  DOUBLE
);

2. Run the `SHOW TABLES` statement to view tables in your default catalog and the schema we set earlier. In the results, you should see the table named **silver** in the **example** schema (database).

In [0]:
SHOW TABLES;

database,tableName,isTemporary
example,silver,false


#### C3.2 Populate and Query the Managed Table
1. Populate the **silver** table with some sample data.

In [0]:
INSERT OVERWRITE silver VALUES
  (23,'40580129','Nicholas Spears','2020-02-01T00:01:58.000+0000',54.0122153343),
  (17,'52804177','Lynn Russell','2020-02-01T00:02:55.000+0000',92.5136468131),
  (37,'65300842','Samuel Hughes','2020-02-01T00:08:58.000+0000',52.1354807863),
  (23,'40580129','Nicholas Spears','2020-02-01T00:16:51.000+0000',54.6477014191),
  (17,'52804177','Lynn Russell','2020-02-01T00:18:08.000+0000',95.033344842),
  (37,'65300842','Samuel Hughes','2020-02-01T00:23:58.000+0000',57.3391541312),
  (23,'40580129','Nicholas Spears','2020-02-01T00:31:58.000+0000',56.6165053697),
  (17,'52804177','Lynn Russell','2020-02-01T00:32:56.000+0000',94.8134313932),
  (37,'65300842','Samuel Hughes','2020-02-01T00:38:54.000+0000',56.2469995332),
  (23,'40580129','Nicholas Spears','2020-02-01T00:46:57.000+0000',54.8372685558),
  (23,'40580129',NULL,'2020-02-01T00:01:58.000+0000',54.0122153343),
  (17,'52804177',NULL,'2020-02-01T00:02:55.000+0000',92.5136468131),
  (37,'65300842',NULL,'2020-02-01T00:08:58.000+0000',1000052.1354807863),
  (23,'40580129',NULL,'2020-02-01T00:16:51.000+0000',54.6477014191),
  (17,'52804177','Lynn Russell','2020-02-01T00:18:08.000+0000',9),
  (37,'65300842','Samuel Hughes','2020-02-01T00:23:58.000+0000',7),
  (23,'40580129','Nicholas Spears','2020-02-01T00:31:58.000+0000',6),
  (17,'52804177','Lynn Russell','2020-02-01T00:32:56.000+0000',5),
  (37,'65300842','Samuel Hughes','2020-02-01T00:38:54.000+0000',9000),
  (23,'40580129','Nicholas Spears','2020-02-01T00:46:57.000+0000',66),
  (23,'40580129',NULL,'2020-02-01T00:01:58.000+0000',54.0122153343),
  (17,'52804177',NULL,'2020-02-01T00:02:55.000+0000',92.5136468131),
  (37,'65300842',NULL,'2020-02-01T00:08:58.000+0000',1000052.1354807863),
  (23,'40580129',NULL,'2020-02-01T00:16:51.000+0000',54.6477014191),
  (17,'52804177','Lynn Russell','2020-02-01T00:18:08.000+0000',98),
  (37,'65300842','Samuel Hughes','2020-02-01T00:23:58.000+0000',90),
  (23,'40580129','Nicholas Spears','2020-02-01T00:31:58.000+0000',60),
  (17,'52804177','Lynn Russell','2020-02-01T00:32:56.000+0000',50),
  (37,'65300842','Samuel Hughes','2020-02-01T00:38:54.000+0000',30),
  (23,'40580129','Nicholas Spears','2020-02-01T00:46:57.000+0000',80);

num_affected_rows,num_inserted_rows
30,30


2. Execute queries on the created table to examine its contents, ensuring that the data is correctly stored and accessible for analysis purposes.

In [0]:
-- View the rows inside the silver table
SELECT * 
FROM silver;

device_id,mrn,name,time,heartrate
23,40580129,Nicholas Spears,2020-02-01T00:01:58Z,54.0122153343
17,52804177,Lynn Russell,2020-02-01T00:02:55Z,92.5136468131
37,65300842,Samuel Hughes,2020-02-01T00:08:58Z,52.1354807863
23,40580129,Nicholas Spears,2020-02-01T00:16:51Z,54.6477014191
17,52804177,Lynn Russell,2020-02-01T00:18:08Z,95.033344842
37,65300842,Samuel Hughes,2020-02-01T00:23:58Z,57.3391541312
23,40580129,Nicholas Spears,2020-02-01T00:31:58Z,56.6165053697
17,52804177,Lynn Russell,2020-02-01T00:32:56Z,94.8134313932
37,65300842,Samuel Hughes,2020-02-01T00:38:54Z,56.2469995332
23,40580129,Nicholas Spears,2020-02-01T00:46:57Z,54.8372685558


### C4. Creating and Managing Views

1. Let's create a **vw_gold** view that presents a processed version of the **silver** table data by averaging heart rate data per patient on a daily basis. In the following cell, we will:
    - Create the view
    - Query the view

In [0]:
-- Create a gold view
CREATE OR REPLACE VIEW vw_gold AS (
  SELECT 
    mrn, 
    name, 
    MEAN(heartrate) 
    avg_heartrate, 
    DATE_TRUNC("DD", time) AS date
  FROM silver
  GROUP BY mrn, name, DATE_TRUNC("DD", time)
);


-- View the rows inside the gold view
SELECT * 
FROM vw_gold;

mrn,name,avg_heartrate,date
52804177,Lynn Russell,63.48006043547143,2020-02-01T00:00:00Z
40580129,null,54.329958376700006,2020-02-01T00:00:00Z
65300842,Samuel Hughes,1327.5316620643857,2020-02-01T00:00:00Z
52804177,null,92.5136468131,2020-02-01T00:00:00Z
40580129,Nicholas Spears,54.0142113348625,2020-02-01T00:00:00Z
65300842,null,1000052.1354807863,2020-02-01T00:00:00Z


2. Use the `SHOW VIEWS` statement to display all available views. Run the cell and view the results. Notice that we have a view named **vw_gold**. The view is neither temporary nor materialized.

In [0]:
SHOW VIEWS;

namespace,viewName,isTemporary,isMaterialized,isMetric
example,vw_gold,false,false,false


### C5. Creating, Managing and Executing User-Defined Functions
1. A User-Defined Function (UDF) in SQL is a feature that allows you to extend the native capabilities of SQL. It enables you to define your business logic as reusable functions that extend the vocabulary of SQL for transforming or masking data and reuse it across your applications. User-defined functions are contained by schemas as well. For this example, we'll set up simple function that masks all but the last two characters of a string.

In [0]:
-- Create a custom function to mask a string value except for 2 left-most characters
CREATE OR REPLACE FUNCTION dbacademy_mask(x STRING)
  RETURNS STRING
  RETURN CONCAT(LEFT(x, 2) , REPEAT("*", LENGTH(x) - 2));

2. Manually view the newly created function by completing the following steps:

   - Select the **Catalog** icon (3rd one in icon list) in the left vertical bar (directly under **File** menu).

   - Expand your unique catalog name.

   - Expand the **example** schema we previously created. Notice that the schema now contains **Tables** and **Functions**.

   - Expand **Functions** (you might have to refresh your schema).
   
   -  Notice that the newly created function, **dbacademy_mask**, is available in your schema.




3. Let's see how the function we just defined works. Note that you can expand the table by dragging the border to the right if the output is truncated.

In [0]:
-- Run the custom function to verify its output
SELECT dbacademy_mask('sensitive data') AS data;

data
se************


### D. Catalog Explorer
All of the data objects we have created are available to us in the Data Explorer. We can view and change permissions, change object ownership, examine lineage, and a whole lot more.



#### D1. Using the Catalog Explorer
1. Open the Catalog Explorer by right-clicking on **Catalog** in the far left sidebar menu and opening the link in a new tab. This will allow you to keep these instructions open.

1. Click on your catalog's name. 

    You will see a list of at least five schemas in the catalog: 
    - **default** - this schema is created when the catalog is created and can be dropped, if needed
    - **dmguc** - created by the *Classroom Setup* script we ran at the beginning of the notebook (**dmguc** is short for *Data Management and Governance with Unity Catalog*)
    - **example** - we created this schema above
    - **information_schema** - this schema is created when the catalog is created and contains a wealth of metadata. We will talk more about this schema in a future lesson
    - **other** - created by the *Classroom Setup* script.

#### D2. Delete a Schema
Let's delete the **other** schema. 
1. Under **Your** Catalog, click the **other** schema in the left navigation bar. 

2. Select the three dots to the right of the schema name.

3. Select **Open in Catalog Explorer**. 

4. In the upper-right corner, click the three dots. Then you can select **Delete** to delete the **other** schema. 

5. Click the warning to accept.

6. Close the Catalog Explorer browser.

You can also drop a schema with code:
```
DROP SCHEMA IF EXISTS other;
```

In [0]:
-- If didn't DROP the 'other' schema via the User Interface, you can use the below code to do the same thing

DROP SCHEMA IF EXISTS other;

#### D3. Add AI Generated Comments
Now, lets look at the *example* schema we created:

1. Click **example** schema in the left navigation bar and select the three dots.

1. Select **Open in Catalog Explorer** to view the details for this schema.

1. Note the two data objects in the schema, the **silver** table and the **vw_gold** view. 

1. Click the **silver** table name.

1. We see the columns we defined for this table. Note the button labeled **AI Generate**. We can use this to generate comments for our columns. 

1. Click **AI generate**.

1. The Data Intelligence Platform proposes comments for each column in our table. Click the check next to the first comment to accept it.

1. We also have an AI suggested description for the table. Click **Accept**. 

1. Leave the browser open.

#### D4. Explore Table Information
1. There are tabs along the top where we can view and manage metadata about this table:
      - **Overview** - On the right, we have information about the table, tags, its description, and we can add a row filtering function if we wish. We will talk about this in a future lesson. On the left, we get information about each column in the table

      - **Sample data** - This tab gives us the first rows of the table, so we can see the data contained within the table.

      - **Details** - We get the same information here as we would get by running **`DESCRIBE EXTENDED silver`**.

      - **Permissions** - The UI gives us the ability to manage table permissions. We can **`GRANT`** and **`REVOKE`** permissions here. We will talk about doing this programmatically in a future lesson.

      - **History** - The Delta Lake transaction log is on this tab. We can get this programmatically by running **`DESCRIBE HISTORY silver`**.
      
      - **Lineage** - It is very helpful to see table lineage. Click **`See lineage graph`** to see both our *silver* table and the *gold* view. Note that the view gets its data from the *silver* table. Click the "X" in the upper-right corner to close the window.

      - **Insights** - The Databricks Data Intelligence Platform provides these insights, so we can see how our data object is being used.

      - **Quality** - This tab gives us the ability to monitor this table for data quality. Let's talk more about this next.

### E. Dropping a Managed Table
1. Because the table we created is a managed table, when it is dropped, the data we added to it is also deleted.

In [0]:
DROP TABLE silver;

## Conclusion
In this demo, we explored the process of upgrading tables to Unity Catalog, focusing on the three-level namespace concept. We created and managed catalogs, schemas, tables, views, and user-defined functions, demonstrating the execution of SQL commands for each task. We ensured correct storage and accessibility of data objects, created and populated a managed table and view, and defined a user-defined function to enhance SQL capabilities within the Unity Catalog environment. This comprehensive approach provided a structured and efficient method to manage data in the metastore, leveraging Unity Catalog's advanced features.


&copy; 2025 Databricks, Inc. All rights reserved. Apache, Apache Spark, Spark, the Spark Logo, Apache Iceberg, Iceberg, and the Apache Iceberg logo are trademarks of the <a href="https://www.apache.org/" target="blank">Apache Software Foundation</a>.<br/>
<br/><a href="https://databricks.com/privacy-policy" target="blank">Privacy Policy</a> | 
<a href="https://databricks.com/terms-of-use" target="blank">Terms of Use</a> | 
<a href="https://help.databricks.com/" target="blank">Support</a>
